In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

## Data Exploration
- Let's extract the data first and concat the two json file into single csv file. Save the csv file
- Data exploration afterwards

In [2]:
# df = pd.read_json('gumtree.json')
# df2 = pd.read_json('remaininggumtree.json')
# data = pd.concat([df,df2])
# #Saving it as a final_data.csv
# data.to_csv('final_data.csv',index=False)

In [3]:
dataset = pd.read_csv('final_data.csv')

In [4]:
#Let's check out the coloumn names
dataset.columns #All the coloumns here are important so we won't be deleting it. 

Index(['Year', 'Body type', 'Mileage', 'Transmission', 'Brochure Engine size',
       'Fuel type', 'Urban mpg', 'Extra Urban mpg', 'Fuel Consumption',
       'Insurance Group', 'CO2 emissions', 'Euro Emissions', 'Engine Power',
       'Engine size', 'Acceleration (0-62mph)', 'Top Speed', 'Seats', 'Doors',
       'Colour', 'Fuel Capacity', 'Luggage Capacity (seats up)', 'Price',
       'Name', 'Location'],
      dtype='object')

In [5]:
#However some of the coloumns needs some renaming to be done 
#The reason we are doing this is because we want
dataset.rename({'Acceleration (0-62mph)':'Acceleration(0-62mph)','Luggage Capacity (seats up)':'Luggage Capacity(seats_up)'},inplace=True)

In [6]:
#Checking if there is any duplicate values in the dataset
#Let's find out duplicate values
dataset.duplicated().sum() #There are alltogether 300 values that are duplicate
#Let's see what those duplicate values are
duplicates = dataset[dataset.duplicated()]

In [7]:
#duplicates.to_csv('duplicates.csv',index=False) #Exporting to csv to further analyse the data properly.(Not necessary)

In [8]:
#Let's remove the duplciates from the main dataset
final_dataset = dataset.drop_duplicates()

In [9]:
final_dataset #We sucessfully removed all the 300 duplicates from the data. 
#Before the data dimanesion was 1078 rows × 24 columns and now after removing 300 duplicate values, it is 778 rows × 24 columns

,Year,Body type,Mileage,Transmission,Brochure Engine size,Fuel type,Urban mpg,Extra Urban mpg,Fuel Consumption,Insurance Group,...,Acceleration (0-62mph),Top Speed,Seats,Doors,Colour,Fuel Capacity,Luggage Capacity (seats up),Price,Name,Location
0,2017,Other,"31,000miles",Manual,1.0L,Petrol,42.8mpg,60.1mpg,52.3mpg,13E,...,11.8seconds,115mph,5.0,5.0,Blue,52litres,310litres,"£9,495.00",2017 Ford Ecosport 1.0 T EcoBoost Titanium S 5...,"St Georges, Shropshire"
1,2008,Hatchback,"116,264miles",Manual,1.3L,Diesel,49.6mpg,74.3mpg,62.8mpg,5E,...,13.6seconds,101mph,5.0,5.0,Blue,45litres,285litres,"£1,195.00","Vauxhall, CORSA, Hatchback, 2008, Manual, 1248...","Durham, County Durham"
2,2009,Hatchback,"24,000miles",Manual,1.2L,Petrol,38.2mpg,55.4mpg,47.9mpg,11E,...,13.5seconds,104mph,5.0,3.0,Other,46litres,251litres,"£2,195.00","Nissan, MICRA, Acenta Plus, Hatchback, 2009, M...","Sunderland, Tyne and Wear"
3,2006,Hatchback,"112,349miles",Manual,2.0L,Petrol,26.2mpg,45.6mpg,35.8mpg,28E,...,7.1seconds,146mph,5.0,3.0,Red,55litres,350litres,"£4,495.00",Audi A3 2.0T FSI Quattro</h1,"Moira, County Armagh"
4,2015,Hatchback,"42,187miles",Automatic,1.4L,Petrol,36.7mpg,57.6mpg,47.9mpg,6E,...,13.9seconds,106mph,5.0,5.0,Other,45litres,285litres,"£6,980.00",2015 Vauxhall Corsa SE AUTO Hatchback Petrol A...,"Chester, Cheshire"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,2015,Hatchback,"40,000miles",Manual,1.6L,Diesel,65.7mpg,91.1mpg,80.7mpg,19E,...,9.5seconds,124mph,5.0,5.0,Blue,45litres,270litres,"£11,495.00",2015 15 AUDI A1 1.6 SPORTBACK TDI S LINE 5D 11...,"Pontarddulais, Swansea"
1074,2012,Hatchback,"54,000miles",Manual,1.0L,Petrol,50.4mpg,67.3mpg,60.1mpg,7E,...,14.7seconds,99mph,5.0,5.0,Red,45litres,178litres,"£3,495.00",2012 62 SUZUKI SPLASH 1.0 SZ2 5D 68 BHP</h1,"Pontarddulais, Swansea"
1075,2019,Hatchback,"2,373miles",Automatic,1.5L,Diesel,62.8mpg,70.6mpg,67.3mpg,15E,...,11.6seconds,119mph,5.0,5.0,Grey,53litres,514litres,"£17,711.00",2019 Vauxhall Grandland X Grandland X 1.5 Turb...,"Telford, Shropshire"
1076,2016,Hatchback,"47,000miles",Manual,1.2L,Petrol,54.3mpg,76.3mpg,65.7mpg,15E,...,9.6seconds,118mph,5.0,3.0,Black,50litres,285litres,"£6,495.00",2016 Peugeot 208 1.2 PureTech GT Line (s/s) 3d...,"Thrapston, Northamptonshire"


In [10]:
#Let's now count how many missing values are in the dataset
#Since the missing values are just blank in the dataset, let's fill those values with nan 
final_dataset = final_dataset.replace(r'^\s*$',np.nan,regex = True) #Using regex to replace all the white spaces with NaN


In [33]:
#Let's clean coloumns that needs cleaning
def remove_html_tags(string):
    result = re.sub('</h1','',string)
    return result

def remove_emojis(col): #Upon careful looking at the dataset carefully, there is an emoji '💥' that is in dataset s
    col = col.apply(lambda x: x.encode('ascii','ignore').decode('ascii')) #let's remove it. 
    return col

def value_fill(data,colname1,value1,colname2,value2,new_value): #WORK ON THIS
    if data[data[colname1]]==value1:
        data[data[colname2]].replace({value2:new_value},inplace=True)
    return data
        

final_dataset['Name'] = final_dataset['Name'].apply(lambda x : remove_html_tags(x))
final_dataset['Name'] = remove_emojis(final_dataset['Name']) 

In [23]:
empty_brochure = final_dataset[final_dataset['Brochure Engine size'].isna()] #Let's seperate out and create new dataset
                                                                             #that only has empty brochure engine size

In [ ]:
#Looks like there are alot of missing values that we need to handle. The coloumns with missing values are
#Brochure Engine Size
#Urban mpg
#Extra Urban mpg
#Insurance group
#C02 emissions
#Euro Emissions
#Engine Power
#Acceleration
#Top Speed
#Seats
#Doors
#Fuel Capacity
#Luggage Capacity
#Let's work on the missing values one by one. The good thing about missing values in car is that you can simply manually google it

In [23]:
#Brochure engine size
#Before googling the data, we need to check which rows of brochure engine size datas are missing from and which car it belongs to
empty_brochure = final_dataset[final_dataset['Brochure Engine size'].isna()]
# The four missing values for borchure are 
#2008 Toyota Hilux 3.0l Invincible ---> 3.0L
#2011 Volkswagen golf match TDI 1.6 105 PS HatchBack -->1.6L
#2021 Citroen C4 50kWh Shine Plus Auto 5dr HatchBack --Electric
#2021 Nissan Navara Brand new 2121 model Double hatchback-->2.3L

#Looks like some of the empty brochure size is of that of electric cars, so let's clean that aswell

In [25]:
#Let's create a method that can fill on a row data 
def replace_value(col,row,value):
    if col[row]==value:
        

,Year,Body type,Mileage,Transmission,Brochure Engine size,Fuel type,Urban mpg,Extra Urban mpg,Fuel Consumption,Insurance Group,...,Acceleration (0-62mph),Top Speed,Seats,Doors,Colour,Fuel Capacity,Luggage Capacity (seats up),Price,Name,Location
206,2008,Pick Up,"140,000miles",Automatic,NaN,Diesel,27.4mpg,39.8mpg,34mpg,Not Available,...,12seconds,106mph,5.0,NaN,Black,80litres,NaN,"£4,595.00",Toyota Hilux 3.0l Invincible,"Lochgelly, Fife"
208,2011,Hatchback,"105,496miles",Manual,NaN,Diesel,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Blue,NaN,NaN,"£4,295.00",2011 Volkswagen Golf Match TDI 1.6 105 PS Hatc...,"Peterborough, Cambridgeshire"
445,2021,Hatchback,500miles,Automatic,NaN,Electric,NaN,NaN,NaN,22P,...,9seconds,93mph,5.0,5.0,Blue,NaN,380litres,"£29,995.00",2021 Citroen C4 50kWh Shine Plus Auto 5dr Hatc...,"Chiswick, London"
947,2019,Pick Up,66miles,Manual,NaN,Diesel,27.7mpg,45.6mpg,32.5mpg,Not Available,...,11.2seconds,114mph,5.0,NaN,White,73litres,NaN,"£30,995.00",2021 Nissan Navara Brand new 2121 model Doubl...,"Chesterfield, Derbyshire"
